<a href="https://colab.research.google.com/github/google-research/text-to-text-transfer-transfrormer/blob/main/notebooks/t5-deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The T5 Authors

Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
# Copyright 2020 The T5 Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# T5 SavedModel Export and Inference

This notebook guides you through the process of exporting a [T5](https://github.com/google-research/text-to-text-transformer) `SavedModel` for inference. It uses the fine-tuned checkpoints in the [T5 Closed Book QA](https://github.com/google-research/google-research/tree/main/t5_closed_book_qa) repository for the [Natural Questions](https://ai.google.com/research/NaturalQuestions/) task as an example, but the same process will work for any model trained with the `t5` library.

For more general usage of the `t5` library, please see the main [github repo](https://github.com/google-research/text-to-text-transfer-transformer) and fine-tuning [colab notebook](https://goo.gle/t5-colab).



## Install T5 Library

In [2]:
!pip install -q t5
!git clone https://github.com/google-research/google-research.git --depth=1
# Add closed-book qa library to Python path.
%env PYTHONPATH="/content/google-research/:/content/google-research/t5_closed_book_qa:${PYTHONPATH}"

ERROR: Cannot install t5==0.0.0, t5==0.1.0, t5==0.1.1, t5==0.1.2, t5==0.1.3, t5==0.1.4, t5==0.1.5, t5==0.1.6, t5==0.1.7, t5==0.2.0, t5==0.3.0, t5==0.3.2, t5==0.4.0, t5==0.4.1, t5==0.5.0, t5==0.6.0, t5==0.6.1, t5==0.6.2, t5==0.6.3, t5==0.6.4, t5==0.7.0, t5==0.7.1, t5==0.8.0, t5==0.8.1, t5==0.9.0, t5==0.9.1, t5==0.9.2 and t5==0.9.3 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts
Cloning into 'google-research'...
remote: Enumerating objects: 19294, done.
remote: Counting objects: 100% (19294/19294), done.
remote: Compressing objects: 100% (15281/15281), done.


## Export `SavedModel` to local storage

NOTE: This will take a while for XL and XXL.

In [ ]:
MODEL = "small_ssm_nq" #@param["small_ssm_nq", "t5.1.1.xl_ssm_nq", "t5.1.1.xxl_ssm_nq"]

import os

saved_model_dir = f"/content/{MODEL}"

!t5_mesh_transformer \
  --module_import="t5_cbqa.tasks" \
  --model_dir="gs://t5-data/pretrained_models/cbqa/{MODEL}" \
  --use_model_api \
  --mode="export_predict" \
  --export_dir="{saved_model_dir}"

saved_model_path = os.path.join(saved_model_dir, max(os.listdir(saved_model_dir)))

## Load `SavedModel` and create helper functions for inference

NOTE: This will take a while for XL and XXL.

In [ ]:
import tensorflow as tf
import tensorflow_text  # Required to run exported model.

model = tf.saved_model.load(saved_model_path, ["serve"])

def predict_fn(x):
 return model.signatures['serving_default'](tf.constant(x))['outputs'].numpy()

def answer(question):
  return predict_fn([question])[0].decode('utf-8')

## Ask some questions

We must prefix each question with the `nq question:` prompt since T5 is a multitask model.

In [ ]:
for question in ["nq question: where is google's headquarters",
                 "nq question: what is the most populous country in the world",
                 "nq question: name a member of the beatles",
                 "nq question: how many teeth do humans have"]:
    print(answer(question))

## Package in Docker image

```bash
MODEL_NAME=model-name
SAVED_MODEL_PATH=/path/to/export/dir

# Download the TensorFlow Serving Docker image and repo:
docker pull tensorflow/serving:nightly

# First, run a serving image as a daemon:
docker run -d --name serving_base tensorflow/serving:nightly

# Next, copy your `SavedModel` to the container's model folder:
docker cp $SAVED_MODEL_PATH serving_base:/models/$MODEL_NAME

# Now, commit the container that's serving your model:
docker commit --change "ENV MODEL_NAME ${MODEL_NAME}" serving_base $MODEL_NAME

# Finally, save the image to a tar file:
docker save $MODEL_NAME -o $MODEL_NAME.tar

# You can now stop `serving_base`:
docker kill serving_base
```

```bash
docker run -t --rm -p 8501:8501 --name $MODEL_NAME-server $MODEL_NAME &

curl -d '{"inputs": ["nq question: what is the most populous country?"]}' \
    -X POST http://localhost:8501/v1/models/$MODEL_NAME:predict

docker stop $MODEL_NAME-server
```